In [1]:
from constants import *
from obfuscation import *
from run_obfuscation import * 
from data_utils import *
from stereo_utils import *
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle
from pathlib import Path

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [6]:
dataset_files = list(read_dataset_to_obfuscate(f"{ROOT_DIR}/obfuscation/ml-1m"))
original_data = { data_slice:transform_to_obf(data) for data_slice,data in zip(['train','valid','test'],dataset_files)}


In [37]:
data_dir = f"{ROOT_DIR}/obfuscation/ml-1m"
def calculate_dataset_stereo(data_dir,stereo_types=["mean","median"]):
    train_data, valid_data, test_data, inclination_data, user_features, dataset_name  = read_dataset_to_obfuscate(data_dir)
    for stereo_type in stereo_types:
        user_stereo_scores = []
        for user  in user_features["userID"].values:        
            user_data, valid_user_items, user_ff_values, user_stereo_score = prepare_user_to_obf(ff_data=inclination_data,sterotyp_method=stereo_type,train_data=train_data,user=user)
            user_stereo_scores.append(user_stereo_score)
        user_features[f"stereo_{stereo_type}"] = user_stereo_scores
    return user_features
def read_dataset_stereo(data_dir):
    dataset_name = data_dir.split("/")[-1]
    return pd.read_csv(f"{data_dir}/{dataset_name}_user_ster.csv",index_col="userID")

In [38]:
obf_dir = f"{ROOT_DIR}/obfuscation"
data_dirs = !ls {ROOT_DIR}/ob*
dataset = "ml-1m-1000_"
data_dict = {url:f"{obf_dir}/{url}" for url in data_dirs if url.startswith(dataset)} 

In [39]:
obf_dict_datasets={}
original_ster_data = read_dataset_stereo(f"{obf_dir}/ml-1m-1000")
for dset,url in data_dict.items():
    print(f"Proessing: {dset}")
    obf_dict_datasets[dset]=read_dataset_stereo(url)
    obf_dict_datasets[dset][dset]=obf_dict_datasets[dset]
    obf_dict_datasets[dset]=obf_dict_datasets[dset]


Proessing: ml-1m-1000_remove_0.05_ff_inc_ratio_th0.005
Proessing: ml-1m-1000_remove_0.05_ff_mean_th0.005
Proessing: ml-1m-1000_remove_0.05_ff_median_th0.005
Proessing: ml-1m-1000_remove_0.15_ff_inc_ratio_th0.005
Proessing: ml-1m-1000_remove_0.15_ff_mean_th0.005
Proessing: ml-1m-1000_remove_0.15_ff_median_th0.005
Proessing: ml-1m-1000_remove_0.1_ff_inc_ratio_th0.005
Proessing: ml-1m-1000_remove_0.1_ff_mean_th0.005
Proessing: ml-1m-1000_remove_0.1_ff_median_th0.005


In [40]:
obf_dict_datasets['ml-1m-1000_remove_0.05_ff_inc_ratio_th0.005']


,user_ster
userID,
13,7.183099
21,8.066667
22,4.866025
35,1.156737
43,1.630769
...,...
6016,1.625948
6017,1.044025
6024,5.124717


In [19]:
original_ster_data["tag"]="original"

In [21]:
original_ster_data

,mean,median,inc_ratio,tag
userID,,,,
13,0.303122,0.346869,7.250564,original
21,0.227644,0.248092,8.066667,original
22,0.286752,0.322569,4.989201,original
35,-0.019642,-0.015345,1.161631,original
43,0.038867,0.000000,1.630769,original
...,...,...,...,...
6016,0.054987,0.085106,1.619881,original
6017,-0.015751,-0.013514,1.044025,original
6024,0.251602,0.350000,4.550000,original
